In [1]:
from tc_python import *
import itertools as itertool
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def singlePoint(database=["tcfe9"],T=[273],P=[1e5],components=["fe","c"],phases=["bcc"],mole_fractions=[99.02,0.08]):
    """
    Single point equilibrium calculations
    
    ## input: 
        database name: [string],
        Temperature float,
        Pressure float,
        elements: [string], 
        phases [string], if empty all phases from the database are included
        mole fractions [float]
            
    ## output: dictionary {"nps","vs","ws","xiphs","ys", "acs","mus" }
        stable_phases: [string],
        npms: phase fractions [float],
        vpvs: volume fractions of phases [float], 
        ws: weight fractions of elements [float],
        xiphs: mole fractions of elements in phases [float],
        ys: y fractions of elements is phases [float], 
        bineries: binary list of all elements and stable phases [tuple (component,phase)]
        acs: activities of elements with respect to all phases 
        mus: chemical potentials of all components
    """
    with TCPython() as start:
        if not phases:
            system_int = start.select_database_and_elements(database,components)
        else:    
            system_int = start.select_database_and_elements(database,components).without_default_phases()
            for phase in phases:
                system_int.select_phase(phase)
        system = system_int.get_system()
        ticc=time.time()
        calc = system.with_single_equilibrium_calculation()
        for i in range(len(components)-1):
            calc.set_condition(ThermodynamicQuantity.mole_fraction_of_a_component((components[i])), mole_fractions[i])
        calc.set_condition(ThermodynamicQuantity.temperature(), 1723.15)
        calc.set_condition(ThermodynamicQuantity.pressure(), 1e5)
        calc_res = calc.calculate()
        
        stable_phases = calc_res.get_stable_phases()
        volume_fractions,phase_fractions,weight_fractions,xs_in_phases,ys_in_phases,activities,chemical_potentials = \
        [],[],[],[],[],[],[]
        for phase in stable_phases:
            volume_fractions.append(calc_res.get_value_of('vpv({})'.format(phase)))       
            phase_fractions.append(calc_res.get_value_of('npm({})'.format(phase)))
        for element in components:
            weight_fractions.append(calc_res.get_value_of('w({})'.format(element)))
            chemical_potentials.append(calc_res.get_value_of('mu({})'.format(element)))
        binarys = list(itertool.product(stable_phases, components))
        for binary in binarys:
            xs_in_phases.append(calc_res.get_value_of('x({},{})'.format(binary[0], binary[1])))
            try:
                ys_in_phases.append(calc_res.get_value_of('y({},{})'.format(binary[0], binary[1])))
            except Exception as error:
                a=1
                #ys_in_phases.append(-1)
            try:
                activities.append(calc_res.get_value_of('ac({},{})'.format(binary[1], binary[0])))                    
            except Exception as error:
                a=1
                #ys_in_phases.append(-1)
        tocc=time.time()
        print(tocc-ticc)
        return {"stable_phases":stable_phases,"npms":phase_fractions, \
                "vpvs":volume_fractions,"ws":weight_fractions,"xiphs":xs_in_phases, \
                "ys":ys_in_phases,"acs":activities,"mus":chemical_potentials,"binaries":binarys}

In [3]:
help(singlePoint)

Help on function singlePoint in module __main__:

singlePoint(database=['tcfe9'], T=[273], P=[100000.0], components=['fe', 'c'], phases=['bcc'], mole_fractions=[99.02, 0.08])
    Single point equilibrium calculations
    
    ## input: 
        database name: [string],
        Temperature float,
        Pressure float,
        elements: [string], 
        phases [string], if empty all phases from the database are included
        mole fractions [float]
            
    ## output: dictionary {"nps","vs","ws","xiphs","ys", "acs","mus" }
        stable_phases: [string],
        npms: phase fractions [float],
        vpvs: volume fractions of phases [float], 
        ws: weight fractions of elements [float],
        xiphs: mole fractions of elements in phases [float],
        ys: y fractions of elements is phases [float], 
        bineries: binary list of all elements and stable phases [tuple (component,phase)]
        acs: activities of elements with respect to all phases 
        mus: ch

In [4]:
database = "TCFE8"
elements = ["C","Co","N","Ti","W"]
phases = ["liquid", "fcc", "mc_shp", "graphite"]
mole_fractions = [0.42943834995313096,0.1,0.019999999999999993,0.01999999999999999,0.4305616500468691]

In [5]:
tic=time.time()
a=singlePoint(database,1723, 1e5, elements,phases,mole_fractions)
toc = time.time()
print(toc-tic)

0.18500900268554688
18.717572450637817
